In [ ]:
# %%
import torch
torch.cuda.get_device_name(0)

# %%
from transformers import BertTokenizer, BertForMaskedLM, BertModel
from tokenizer import *
import pickle
from torch.utils.data import DataLoader
import os
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from data import help_tokenize, load_paired_data, FunctionDataset_CL, FunctionDataset_CL_Load
from transformers import AdamW
import torch.nn.functional as F
import argparse
import logging
import sys
import time
import data
import pickle
import sys
from datautils.playdata import DatasetBase as DatasetBase
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import argparse

import os
import shutil


def eval(model, data_loader):
    with torch.no_grad():
        avg=[]
        gt=[]
        cons=[]
        eval_iterator = tqdm(data_loader)
        for i, (seq1,seq2,_,mask1,mask2,_) in enumerate(eval_iterator):
            input_ids1, attention_mask1= seq1.cuda(),mask1.cuda()
            input_ids2, attention_mask2= seq2.cuda(),mask2.cuda()

            anchor,pos=0,0

            output1 = model(input_ids=input_ids1, attention_mask=attention_mask1)
            anchor = output1.pooler_output

            output2 = model(input_ids=input_ids2, attention_mask=attention_mask2)
            pos = output2.pooler_output

            ans=0
            for i in range(len(anchor)):    # check every vector of (vA,vB)
                vA=anchor[i:i+1].cuda()  #pos[i]
                sim=[]
                for j in range(len(pos)):
                    vB=pos[j:j+1].cuda()   # pos[j]
                    AB_sim=F.cosine_similarity(vA, vB).item()
                    sim.append(AB_sim)
                    if j!=i:
                        cons.append(AB_sim)
                sim=np.array(sim)
                y=np.argsort(-sim)
                posi=0
                for j in range(len(pos)):
                    if y[j]==i:
                        posi=j+1

                gt.append(sim[i])
                ans+=1/posi
            ans=ans/len(anchor)
            avg.append(ans)
        return np.mean(np.array(avg)), avg

class BinBertModel(BertModel):
    def __init__(self, config, add_pooling_layer=True):
        super().__init__(config)
        self.config = config
        self.embeddings.position_embeddings=self.embeddings.word_embeddings


In [ ]:

model = None
valid_set = None
valid_dataloader = None
model_path = "models/jTrans-finetune"
model = BinBertModel.from_pretrained(model_path)
eval_path = "extract_selected"
print(f"evaluating {eval_path}")

model = nn.DataParallel(model)


In [ ]:
tokenizer = BertTokenizer.from_pretrained("./jtrans_tokenizer")
valid_set = FunctionDataset_CL_Load(tokenizer, eval_path, convert_jump_addr=True)
valid_dataloader = DataLoader(valid_set, batch_size=32, num_workers=32, shuffle=True)



If this step gives out OOM error, or CUDA memory error, consider reduce the data size or evaluate data in chunks

In [ ]:
mrr, avg = eval(model, valid_dataloader)
